In [77]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import os
import datetime

In [49]:
file_list = os.listdir('C:\JanbogoProject\JangbogoProject\data\\2019_data')
path = 'C:\JanbogoProject\JangbogoProject\data\\2019_data'

In [50]:
#너무 커서 안됨
# df_list=[]
# for filename in file_list:
#     df = pd.read_csv(path + '\\' +  filename)
#     df_list.append(df)
# data = pd.concat(df_list, sort=False).reset_index(drop=True) 

In [51]:
#통합데이터에서 가져오기로...
df = pd.read_csv('C:\JanbogoProject\JangbogoProject\develop\\target_pum_kind.csv')

C:\Users\song\anaconda3\envs\JanbogoProject2\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
df1 = df.drop('Unnamed: 0',axis=1)
df1

,SALEDATE,PUM_NM,KIND_NM,DAN_NM,SAN_NM,TOT_QTY,TOT_AMT
0,20160102,건고추,기타건고추,kg,전북 익산시,3.0,33000.0
1,20160102,깻잎,깻단,kg,전남 곡성군,62.0,182550.0
2,20160102,깻잎,깻단,kg,전남 광양시,27.0,179500.0
3,20160102,깻잎,깻잎,kg,전남 곡성군,253.0,4027800.0
4,20160102,깻잎,깻잎,kg,전남 광양시,312.0,4969300.0
...,...,...,...,...,...,...,...
3402537,2020-09-28,호박,애호박,kg,충청북도 옥천군,1486.0,3030200.0
3402538,2020-09-28,호박,애호박,kg,충청북도 음성군,18528.0,55185100.0
3402539,2020-09-28,호박,애호박,kg,충청북도 진천군,10980.0,25190900.0
3402540,2020-09-28,호박,애호박,kg,충청북도 청원군,30008.0,88470500.0


In [53]:
df2 = df1.drop('SAN_NM', axis = 1)
df2

,SALEDATE,PUM_NM,KIND_NM,DAN_NM,TOT_QTY,TOT_AMT
0,20160102,건고추,기타건고추,kg,3.0,33000.0
1,20160102,깻잎,깻단,kg,62.0,182550.0
2,20160102,깻잎,깻단,kg,27.0,179500.0
3,20160102,깻잎,깻잎,kg,253.0,4027800.0
4,20160102,깻잎,깻잎,kg,312.0,4969300.0
...,...,...,...,...,...,...
3402537,2020-09-28,호박,애호박,kg,1486.0,3030200.0
3402538,2020-09-28,호박,애호박,kg,18528.0,55185100.0
3402539,2020-09-28,호박,애호박,kg,10980.0,25190900.0
3402540,2020-09-28,호박,애호박,kg,30008.0,88470500.0


In [54]:
df3 = df2.groupby(['SALEDATE','PUM_NM','KIND_NM','DAN_NM'])['TOT_QTY','TOT_AMT'].sum().reset_index()

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_36168/658758426.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df2.groupby(['SALEDATE','PUM_NM','KIND_NM','DAN_NM'])['TOT_QTY','TOT_AMT'].sum().reset_index()


In [55]:
#날짜 데이터 동일화
df1.SALEDATE = df1.SALEDATE.apply(lambda x: str(x) if '-' not in str(x) else str(x).replace('-',''))

In [56]:
#2019년 데이터만 뽑아내기
df_2019 = df1[df1.SALEDATE.apply(lambda x : True if x[:4] == '2019' else False)]

In [57]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [58]:
joosan_dict = dict()

# 품종별 주산지 mapping
for pum in unique_pum :
    pum_df = df_2019[df_2019['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

# 품종별 주산지 mapping
for pum in unique_kind :
    pum_df = df_2019[df_2019['KIND_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

In [59]:
joosan_dict

{'배추': '전라남도 해남군',
 '무': '제주도 제주시',
 '양파': '전라남도 무안군',
 '건고추': '전남 해남군',
 '마늘': '경상남도 창녕군',
 '대파': '전라남도 진도군',
 '얼갈이배추': '경기도 포천시',
 '양배추': '제주도 제주시',
 '깻잎': '경상남도 밀양시',
 '시금치': '경기도 포천시',
 '미나리': '경기도 시흥시',
 '당근': '제주도 제주시',
 '파프리카': '강원도 철원군',
 '새송이': '충청남도 천안시',
 '팽이버섯': '경상북도 청도군',
 '토마토': '부산 강서구',
 '청상추': '전북 남원시',
 '백다다기': '충청남도 천안시',
 '애호박': '경상남도 진주시',
 '캠벨얼리': '경상북도 상주시',
 '샤인마스캇': '경상북도 김천시'}

### 주산지 위도, 경도 구하기

In [60]:

y = [] #위도
x = [] #경도

Kakao_ServiceKey = open('Kakao_ServiceKey.txt', 'r').read()
# print(Kakao_ServiceKey)
headers = {"Authorization":'KakaoAK ' + Kakao_ServiceKey}
for san in tqdm(joosan_dict.values()) :
    # print(san)
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
    result = json.loads(str(requests.get(url, headers=headers).text))
    # print(result)
    match_first = result['documents'][0]['address']
    y.append(float(match_first['y'])) #위도
    x.append(float(match_first['x'])) #경도
joosan_xy = pd.DataFrame({'SAN_NM' : joosan_dict.values(), 'y' : y, 'x' : x}).reset_index()


100%|██████████| 21/21 [00:01<00:00, 16.31it/s]


In [61]:
# from urllib.parse import urlparse

In [62]:
# Kakao_ServiceKey = open('Kakao_ServiceKey.txt', 'r').read()
# Kakao_ServiceKey
# for san in tqdm(joosan_dict.values()) :
#     url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
#     result = requests.get(urlparse(url).geturl(), headers= {'Authorization' : 'KakaoAK ' + Kakao_ServiceKey}).json()
#     print(result)

In [63]:
#주산지 위도와 경도
joosan_xy

,index,SAN_NM,y,x
0,0,전라남도 해남군,34.573517,126.599270
1,1,제주도 제주시,33.499534,126.531171
2,2,전라남도 무안군,34.990341,126.481705
3,3,전남 해남군,34.573517,126.599270
4,4,경상남도 창녕군,35.544615,128.492168
5,5,전라남도 진도군,34.486829,126.263446
6,6,경기도 포천시,37.894993,127.200333
7,7,제주도 제주시,33.499534,126.531171
8,8,경상남도 밀양시,35.503817,128.746608
9,9,경기도 포천시,37.894993,127.200333


### 농업 기상 관측 지점 데이터

In [64]:

stn_info = pd.read_csv('C:\JanbogoProject\JangbogoProject\data\RDA_SPOT_INFO(2021-10-26)2.csv', header = 2)
# stn_info
stn_info['관측시작일'] = pd.to_datetime(stn_info['관측시작일'])
stn_info = stn_info[stn_info['관측시작일'].dt.year<=2015]

In [65]:
stn_info

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14
...,...,...,...,...,...,...,...,...
205,부산,부산시 강서구,618803A001,35.20937,128.96128,3.0,경남 부산시 강서구 대저1동 2038,2010-12-20
206,인천,옹진군 백령면,409911A001,37.95742,124.71063,11.0,인천광역시 옹진군 백령면 진촌리 2359-4,2014-12-18
207,인천,옹진군 영흥면,409871A001,37.25288,126.46034,11.0,인천광역시 옹진군 영흥면 영흥남로435번길 29,2012-05-26
208,울산,울산시 청량면,689861A001,35.52411,129.26704,51.0,울산광역시 울주군 청량면 청량천변로 211,2009-03-02


### 주산지별로 농업기상관측지점과 거리 기준으로 mapping

In [66]:

y_san = joosan_xy['y']
x_san = joosan_xy['x']
y_stn = stn_info['위도']
x_stn = stn_info['경도']
code_dict = dict()

for y_san, x_san, san_name in zip(joosan_xy['y'], joosan_xy['x'], joosan_xy['SAN_NM']) :
    min_distance = 1000 #임의로 초기값 설정
    for y_stn, x_stn, stn_code in zip(stn_info['위도'], stn_info['경도'], stn_info['지점코드']) :
        distance = ((y_san-y_stn)**2 + (x_san-x_stn)**2)**0.5 #거리
        if distance < min_distance :
            min_distance = distance 
            stn_nearby = str(stn_code)
    code_dict[san_name] = stn_nearby

In [67]:
stn_info[stn_info.지점코드 == code_dict['전라남도 무안군']] # 원래 지점 코드가 5.35E+06 인걸로..

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
101,전라남도,무안군 청계면,5.35E+06,34.96849,126.45608,30.0,전라남도 무안군 청계면 청천리 293-2(무안로 199),2010-12-17


### 농업기상관측지점 및 산지 분포 시각화

In [68]:
stn_location = stn_info[['지점명','위도', '경도']]
san_location = joosan_xy

map = folium.Map(location = [36, 128], zoom_start =7)

# 농업기상관측지점 분포 (파랑)
for index in stn_location.index:
    stn_latitude = stn_location.loc[index,"위도"]
    stn_longtitude = stn_location.loc[index,"경도"]
    stn_tooltip = stn_location.loc[index,'지점명']
    folium.Marker([stn_latitude, stn_longtitude], popup = '('+str(stn_latitude)+', '+str(stn_longtitude)+')', tooltip = stn_tooltip).add_to(map) 

# 산지 분포 (빨강)    
for index in san_location.index:    
    san_latitude = san_location.loc[index,"y"]
    san_longtitude = san_location.loc[index,"x"]
    san_tooltip = san_location.loc[index,'SAN_NM']    
    folium.Marker([san_latitude, san_longtitude], popup = '('+str(san_latitude)+', '+str(san_longtitude)+')', tooltip = san_tooltip, icon = folium.Icon(color = 'red')).add_to(map)
    
map

### 농업기상데이터 API - 월별 일 기본 관측데이터 조회

In [69]:
### https://www.data.go.kr/iim/api/selectAPIAcountView.do
### 농촌진흥청 국립농업과학원_농업기상 기본 관측데이터 조회 

In [70]:
CropWeather_ServiceKey = open('CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2015','2016', '2017', '2018', '2019', '2020']
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

100%|██████████| 17/17 [01:30<00:00,  5.34s/it]


In [71]:
weather

""


In [72]:
error

,year,month,stn_code,url,관측시작일
0,2015,01,536824B002,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2013-05-23
1,2015,02,536824B002,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2013-05-23
2,2015,03,536824B002,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2013-05-23
3,2015,04,536824B002,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2013-05-23
4,2015,05,536824B002,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2013-05-23
...,...,...,...,...,...
1219,2020,08,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1220,2020,09,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1221,2020,10,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20
1222,2020,11,037268B004,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2007-03-20


In [73]:
# 중복 제거
weather = weather.drop_duplicates().reset_index(drop=True)
weather

""


In [74]:
# dtype변환(object --> float)
for col in weather.columns[4:] :
    weather[col] = weather[col].astype(float)

### 전처리 (기상변수로 추가)
##### 30일씩 12쿼터로 나누어서 평균 기온, 평균 습도, 누적 강수량, 이상 기후 누적 일수 등 추가

In [78]:
first_date = datetime.datetime.strptime('2020-09-28', '%Y-%m-%d') - datetime.timedelta(360)
print(first_date)
date_list = [] 
for delta in range(360) :
    date = first_date + datetime.timedelta(days = delta)
    date = datetime.datetime.strftime(date, '%Y-%m-%d')
    date_list.append(date)
date_df = pd.DataFrame({'date' : date_list})
train = df3
train2 = pd.concat([date_df, train], sort = False).reset_index(drop=True) #2015~2020-09-28

2019-10-04 00:00:00


In [79]:
train = pd.read_csv('C:\JanbogoProject\JangbogoProject\data\\train.csv')
train

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,2020-09-24,목요일,1856965.0,1839.0,2055640.0,990.0,2281429.2,990.0,2818.4,19101.0,...,50730.0,4509.0,282212.3,3001.0,313139.7,3426.0,504242.6,3620.0,283196.9,10940.0
1729,2020-09-25,금요일,1880095.5,1789.0,1879261.0,1011.0,2074513.0,955.0,1887.1,23095.0,...,54322.0,4178.0,312214.8,2999.0,362741.0,3357.0,479683.1,3618.0,303779.6,10844.0
1730,2020-09-26,토요일,1661090.9,1760.0,1709385.7,1075.0,2089081.2,961.0,959.0,22510.0,...,61213.0,3770.0,327395.8,3065.0,390361.2,3092.0,521493.8,3691.0,313295.7,10636.0
1731,2020-09-27,일요일,25396.0,3066.0,38222.0,1139.0,18240.0,1056.0,60.0,22333.0,...,144.0,4076.0,285.0,3707.0,2464.0,3252.0,21717.0,3567.0,9734.0,10699.0


In [80]:
df3.SALEDATE = pd.to_datetime(df['SALEDATE'])

df3
# df.Date.dt.strftime('%Y%m%d').astype(int)
# datetime.datetime.strftime(date, '%Y-%m-%d')

,SALEDATE,PUM_NM,KIND_NM,DAN_NM,TOT_QTY,TOT_AMT
0,1970-01-01 00:00:00.020160102,건고추,기타건고추,kg,3.00,3.300000e+04
1,1970-01-01 00:00:00.020160102,깻잎,깻단,kg,89.00,3.620500e+05
2,1970-01-01 00:00:00.020160102,깻잎,깻잎,kg,565.00,8.997100e+06
3,1970-01-01 00:00:00.020160102,깻잎,깻잎(일반),kg,3376.90,4.681009e+07
4,1970-01-01 00:00:00.020160102,깻잎,깻잎순,kg,344.00,1.764600e+06
...,...,...,...,...,...,...
165125,1970-01-01 00:00:00.020170123,팽이버섯,황금송이,kg,140.00,4.280000e+05
165126,1970-01-01 00:00:00.020170123,포도,샤인마스캇,.,116.00,1.526000e+06
165127,1970-01-01 00:00:00.020170123,포도,샤인마스캇,kg,87812.20,1.091752e+09
165128,1970-01-01 00:00:00.020170123,포도,캠벨얼리,kg,669658.44,2.598334e+09
